In [ ]:
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
from sklearn.svm import SVC  # sckikit-learn
import warnings; warnings.simplefilter('ignore')
from statsmodels.tsa.stattools import adfuller
import configparser as cp

In [ ]:
ktb = pd.read_csv('./csv/KTB_1m_1d.csv')
lktb = pd.read_csv('./csv/LKTB_1m_1d.csv')
k200 = pd.read_csv('./csv/K200_1m_1d.csv')

In [ ]:
data = pd.DataFrame()
data['Date'] = pd.to_datetime(ktb['DATE'] + ' ' + ktb['TIME'])
data['KTB'] = ktb['CLOSE']
data['LKTB'] = lktb['CLOSE']
data['K200'] = k200['CLOSE']
data.set_index('Date',inplace=True)
data

In [ ]:
returns = np.log(data / data.shift(1)).dropna()
returns.head()

In [ ]:
adfuller(data['KTB']) #test for stationarity of time series

In [ ]:
adfuller(returns['KTB'])

In [ ]:
data.normalize().iplot(kind='line')

In [ ]:
returns.iplot(kind='histogram', subplots=True)

In [ ]:
# lagging된 data를 구하기 위한 함수 : df
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(data[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)  # defines the column name
        # creates the lagged data column with directional values
        df[col] = df[ric].shift(lag)
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

In [ ]:
rics=[
    'KTB',
    'LKTB',
    'K200'
]
dfs = {}
lags = 5
for ric in rics:
    df, cols = add_lags(returns, ric, lags)
    dfs[ric] = df

In [ ]:
print(cols)
print(dfs.keys())
dfs['LKTB']

In [ ]:
# SVM (Support Vector Machine) 모델에 래깅한 5개 수익율의 부호를 x로 하고 해당 시점의 수익율의 부호를 y로 하여 학습
# 학습후 다시 그 데이터(래깅한 5개 수익율의 부호)를 입력으로 하여 predict -> 오버피팅 이지만 일단 해본다.
for ric in rics:
    model = SVC(C=100) # the ML model
    df = dfs[ric].copy()  # getting data for the RIC
    model.fit(np.sign(df[cols]), np.sign(df[ric]))  # model fitting
    dfs[ric]['position'] = model.predict(np.sign(df[cols]))  # prediction

In [ ]:
for ric in rics:
    print('{:10} | {}'.format(ric, dfs[ric]['position'].values[:12])) # predict한 것의 최초 12개씩을 보면

In [ ]:
# 예측한 포지션(롱or숏)에 해당시점의 수익율을 각각 곱해서 전략적용 각 시점 수익율을 구한다.
for ric in rics:
    dfs[ric]['strategy'] = dfs[ric]['position'] * dfs[ric][ric] 

In [ ]:
# 전략적용 각 시점 누적 수익율 plotting. 학습data로 예측을 했으니 당연히 잘나옴.
for ric in rics:
    dfs[ric][[ric, 'strategy']].cumsum().apply(np.exp).iplot()

In [ ]:
# 문제점 해결을 위해 train / test dataset을 나눈다.
split = int(len(data)/2)

# train / test 데이터가 구별되도록 visualize 한다.
vspan = [{'x0': data.index[0], 'x1': data.index[split], 'color': 'green', 'fill': True, 'opacity': .2},
        {'x0': data.index[split], 'x1': data.index[-1], 'color': 'red', 'fill': True, 'opacity': .2}]

data.normalize().iplot(vspan=vspan)

In [ ]:
# train dataset으로 학습하고 나온 pred(롱/숏)에 해당수익율(test_y)를 곱해서 전략적용 수익율을 구함
res = {}
for ric in rics:
    model = SVC(C=100) # the ML model
    df = dfs[ric].copy()  # getting data for the RIC
    split = int(len(df) / 2)
    train_x = np.sign(df[cols]).iloc[:split]
    train_y = np.sign(df[ric]).iloc[:split]
    test_x = np.sign(df[cols]).iloc[split:]
    test_y = df[ric].iloc[split:]
    model.fit(train_x, train_y)  # model fitting
    pred = model.predict(test_x)  # prediction
    strat = pred * test_y
    res[ric] = pd.DataFrame({ric: test_y,
                             'pred': pred,
                             'strategy': strat})

In [ ]:
res['KTB'].head()

In [ ]:
# 시점별 누적수익율 plotting.... not bad!??!
for ric in rics:
    res[ric][[ric, 'strategy']].cumsum().apply(np.exp).iplot()